# RecSys M 2021 - Exercise 3 Template

The aims of this exercise are:
 - Explore a different recommendation dataset
 - Develop and evaluate baseline recommender systems
 - Implement hybrid recommender models
 - Explore diversification issues in recommender systems
 - Revise other material from the lectures.

As usual, there is a corresponding Quiz on Moodle for this Exercise, which should be answered as you proceed. For more details, see the Exercise 3 specification.



# Part-Pre. Preparation 

## Pre 1. Setup Block

This exercise will use the [Goodreads]() dataset for books. These blocks setup the data files, Python etc.

In [1]:
!rm -rf ratings* books* to_read* test*

!curl -o ratings.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-ratings.csv" 
!curl -o books.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-books.csv"
!curl -o to_read.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-to_read.csv"
!curl -o test.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-test.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7631k  100 7631k    0     0  5776k      0  0:00:01  0:00:01 --:--:-- 5781k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2366k  100 2366k    0     0  2882k      0 --:--:-- --:--:-- --:--:-- 2878k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7581k  100 7581k    0     0  7529k      0  0:00:01  0:00:01 --:--:-- 7529k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1895k  100 1895k    0     0  2248k      0 --:--:-- --:--:-- --:--:-- 2245k


In [2]:
#Standard setup
import pandas as pd
import numpy as np
import torch
!pip install git+https://github.com/cmacdonald/spotlight.git@master#egg=spotlight
from spotlight.interactions import Interactions
SEED=20
BPRMF=None

  Cloning https://github.com/cmacdonald/spotlight.git (to revision master) to /tmp/pip-install-tgwk0zae/spotlight_eb3b0846790a4106af307f48b55fda70
  Running command git clone -q https://github.com/cmacdonald/spotlight.git /tmp/pip-install-tgwk0zae/spotlight_eb3b0846790a4106af307f48b55fda70
  Created wheel for spotlight: filename=spotlight-0.1.6-py3-none-any.whl size=34106 sha256=b8e1d2883f4b2421cfbd85b226e505383d2008f453b98a8e83167b489e7e63fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-w18atoos/wheels/1c/2a/31/d187173520bc800643df4e3d1f97dee21d2133ba41085704ed
Successfully built spotlight


## Pre 2. Data Preparation

Let's load the dataset into dataframes.

In [3]:
#load in the csv files
ratings_df = pd.read_csv("ratings.csv")
books_df = pd.read_csv("books.csv")
to_read_df = pd.read_csv("to_read.csv")
test = pd.read_csv("test.csv")

In [4]:
#cut down the number of items and users
counts=ratings_df[ratings_df["book_id"] < 2000].groupby(["book_id"]).count().reset_index()
valid_books=counts[counts["user_id"] >= 10][["book_id"]]

books_df = books_df.merge(valid_books, on="book_id")
ratings_df = ratings_df[ratings_df["user_id"] < 2000].merge(valid_books, on="book_id")
to_read_df = to_read_df[to_read_df["user_id"] < 2000].merge(valid_books, on="book_id")
test = test[test["user_id"] < 2000].merge(valid_books, on="book_id")


#stringify the id columns
def str_col(df):
  if "user_id" in df.columns:
    df["user_id"] = "u" + df.user_id.astype(str)
  if "book_id" in df.columns:
    df["book_id"] = "b" + df.book_id.astype(str)

str_col(books_df)
str_col(ratings_df)
str_col(to_read_df)
str_col(test)

Here we construct the Interactions objects from `ratings.csv`, `to_read.csv` and `test.csv`. We manually specify the num_users and num_items parameters to all Interactions objects, in case the test set differs from your training sets.

In [5]:
from collections import defaultdict
from itertools import count

from spotlight.cross_validation import random_train_test_split

iid_map = defaultdict(count().__next__)


rating_iids = np.array([iid_map[iid] for iid in ratings_df["book_id"].values], dtype = np.int32)
test_iids = np.array([iid_map[iid] for iid in test["book_id"].values], dtype = np.int32)
toread_iids = np.array([iid_map[iid] for iid in to_read_df["book_id"].values], dtype = np.int32)


uid_map = defaultdict(count().__next__)
test_uids = np.array([uid_map[uid] for uid in test["user_id"].values], dtype = np.int32)
rating_uids = np.array([uid_map[uid] for uid in ratings_df["user_id"].values], dtype = np.int32)
toread_uids = np.array([uid_map[iid] for iid in to_read_df["user_id"].values], dtype = np.int32)


uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}


rating_dataset = Interactions(user_ids=rating_uids,
                               item_ids=rating_iids,
                               ratings=ratings_df["rating"].values,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

toread_dataset = Interactions(user_ids=toread_uids,
                               item_ids=toread_iids,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

test_dataset = Interactions(user_ids=test_uids,
                               item_ids=test_iids,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

print(rating_dataset)
print(toread_dataset)
print(test_dataset)

#here we define the validation set
toread_dataset_train, validation = random_train_test_split(toread_dataset, random_state=np.random.RandomState(SEED))

num_items = test_dataset.num_items
num_users = test_dataset.num_users

<Interactions dataset (1999 users x 1826 items x 124762 interactions)>
<Interactions dataset (1999 users x 1826 items x 135615 interactions)>
<Interactions dataset (1999 users x 1826 items x 33917 interactions)>


Finally, this is some utility code that we will use in the exercise.

In [6]:
def getAuthorTitle(iid):
  bookid = iid_rev_map[iid]
  row = books_df[books_df.book_id == bookid]
  return row.iloc[0]["authors"] + " / " + row.iloc[0]["title"]

print("iid 0: " + getAuthorTitle(0) )

iid 0: Carlos Ruiz Zafón, Lucia Graves / The Shadow of the Wind (The Cemetery of Forgotten Books,  #1)


## Pre 3. Example Code

To evaluate some of your hand-implemented recommender systems (e.g. Q1, Q4), you will need to instantiate objects that match the specification of a Spotlight model, which `mrr_score()` etc. expects.


Here is an example recommender object that returns 0 for each item, regardless of user.

In [7]:
from spotlight.evaluation import mrr_score, precision_recall_score

class dummymodel:
  
  def __init__(self, numitems):
    self.predictions=np.zeros(numitems)
  
  #uid is the user we are requesting recommendations for;
  #returns an array of scores, one for each item
  def predict(self, uid):
    #this model returns all zeros, regardless of userid
    return( self.predictions )

#lets evaluate how the effeciveness of dummymodel

print(mrr_score(dummymodel(num_items), test_dataset, train=rating_dataset, k=100).mean())
#as expected, a recommendation model that gives 0 scores for all items obtains a MRR score of 0

0.0


In [8]:
#note that mrr_score() displays a progress bar if you set verbose=True
print(mrr_score(dummymodel(num_items), test_dataset, train=rating_dataset, k=100, verbose=True).mean())


1999it [00:00, 2881.60it/s]

0.0


# Part-A. Combination of Recommendation Models

## Task 1. Explicit & Implicit Matrix Factorisation Models

Create and train three matrix factorisation systems:
 - "EMF": explicit MF, trained on the ratings Interactions object (`rating_dataset`)
 - "IMF": implicit MF, trained on the toread_dataset Interactions object (`toread_dataset_train`)
 - "BPRMF": implicit MF with the BPR loss function (`loss='bpr'`), trained on the toread_dataset Interactions object (`toread_dataset_train`)

Use a variable of the same name for these models, as we will use some of them later (e.g. `BPRMF`).
  
In all cases, you must use the standard initialisation arguments, i.e. 
`n_iter=10, embedding_dim=32, use_cuda=False, random_state=np.random.RandomState(SEED)`.
 
Evaluate each of these models in terms of Mean Reciprocal Rank on the test set. MRR can be obtained using:
```python
mrr_score(X, test_dataset, train=rating_dataset, k=100, verbose=True).mean())
```
where X is an instance of a Spotlight model. Do NOT change the `k` or `train` arguments.

In [9]:
# Add your solution here
# EMF

from spotlight.factorization.explicit import ExplicitFactorizationModel

EMF =  ExplicitFactorizationModel(n_iter=10,
                                    embedding_dim=32,
                                    use_cuda=False,
                                    random_state=np.random.RandomState(SEED)
)
EMF.fit(rating_dataset, verbose=True)

mrr_score(EMF, test_dataset, train=rating_dataset, k=100, verbose=True).mean()

Epoch 0: loss 3.8710271667261593
Epoch 1: loss 0.7940810446123607
Epoch 2: loss 0.6382512643200452
Epoch 3: loss 0.5217335281557725
Epoch 4: loss 0.44844855655167926
Epoch 5: loss 0.40543351120880394
Epoch 6: loss 0.3823863151254224
Epoch 7: loss 0.36336620252762664
Epoch 8: loss 0.35137936695799477


96it [00:00, 956.31it/s]

Epoch 9: loss 0.3396692546237199


1999it [00:02, 994.58it/s]


0.05898399982013507

In [10]:
# IMF

from spotlight.factorization.implicit import ImplicitFactorizationModel

IMF =  ImplicitFactorizationModel(n_iter=10,
                                    embedding_dim=32,
                                    use_cuda=False,
                                    random_state=np.random.RandomState(SEED)
)
IMF.fit(toread_dataset_train, verbose=True)

mrr_score(IMF, test_dataset, train=rating_dataset, k=100, verbose=True).mean()

Epoch 0: loss 0.7677980539090229
Epoch 1: loss 0.53877861825925
Epoch 2: loss 0.47017199658560305
Epoch 3: loss 0.428322009882837
Epoch 4: loss 0.39839018825090156
Epoch 5: loss 0.368275504770144
Epoch 6: loss 0.3473479778699155
Epoch 7: loss 0.32980164804689166
Epoch 8: loss 0.31870100696413023


106it [00:00, 1052.57it/s]

Epoch 9: loss 0.3048194432103971


1999it [00:02, 987.98it/s]


0.3299315791285401

In [11]:
# BPRMF

BPRMF =  ImplicitFactorizationModel(n_iter=10,
                                    embedding_dim=32,
                                    use_cuda=False,
                                    loss='bpr',
                                    random_state=np.random.RandomState(SEED)
)
BPRMF.fit(toread_dataset_train, verbose=True)

mrr_score(BPRMF, test_dataset, train=rating_dataset, k=100, verbose=True).mean()

Epoch 0: loss 0.33895447579616644
Epoch 1: loss 0.19644999289709442
Epoch 2: loss 0.15870640168563938
Epoch 3: loss 0.14147728193059284
Epoch 4: loss 0.132827276100387
Epoch 5: loss 0.12213623321632731
Epoch 6: loss 0.11668406535853755
Epoch 7: loss 0.11047121562626001
Epoch 8: loss 0.10888675406996934


88it [00:00, 876.72it/s]

Epoch 9: loss 0.10400472129782978


1999it [00:02, 981.20it/s]


0.4076771464674879

## Task 2. Hybrid Model

In this task, you are expected to create new hybrid recommendation models that 
combine the two models in Task 1, namely IMF and BPRMF. 

(a) Linearly combine the *scores* from IMF and BPRMF.  Normalise both input scores into the range 0..1 using [sklearn's minmax_scale() function](
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html) before combining them.

(b) Apply a pipelining recommender, where the top 100 items are obtained from IMF and re-ranked using the scores of BPRMF. Items not returned by IMF get a score of 0.

To implement these hybrid models, you should create new classes that abide by the Spotlight model contract (namely, it has a `predict(self, uid)` function that returns a score for *all* items). 

Evaluate each model in terms of MRR. How many users are improved, how many are degraded compared to the BPRMF baseline?

Finally, pass your instantiated model object to the `test_Hybrid_a()` (for (a)) or `test_Hybrid_b()` (for (b)) functions, as appropriate, and record the results in the quiz. For example, if your model for (b) is called `pipeline`, then you would run:
```python
test_Hybrid_b(pipeline)
```

You now have sufficient information to answer the Task 2 quiz questions.

In [12]:
def test_Hybrid_a(combsumObj):
  for i, u in enumerate([5, 20]):
    print("Hybrid a test case %d" % i)
    print(np.count_nonzero(combsumObj.predict(u) > 1))

def test_Hybrid_b(pipeObj):
  for i, iid in enumerate([3, 0]):
    print("Hybrid b test case %d" % i)
    print(pipeObj.predict(0)[iid])



In [13]:
# Add your solutions here and evaluate them
# linearModel

from sklearn.preprocessing import minmax_scale

class linearModel:
  def __init__(self, IMF = IMF, BPRMF = BPRMF):
    self.imf = IMF
    self.bprmf = BPRMF

  def predict(self, uid):
    self.prediction = minmax_scale(self.imf.predict(uid)) + minmax_scale(self.bprmf.predict(uid))
    return self.prediction
    
mrr_score(linearModel(), test_dataset, train=rating_dataset, k=100, verbose=True).mean()

1999it [00:04, 467.20it/s]


0.41010939818893816

In [14]:
linear_check = mrr_score(linearModel(), test_dataset, train=rating_dataset, k=100, verbose=True) - mrr_score(BPRMF, test_dataset, train=rating_dataset, k=100, verbose=True)
print("%d users have an improved RR and %d users have an degraded one." %(np.count_nonzero(linear_check>0), np.count_nonzero(linear_check<0)))

1999it [00:04, 458.84it/s]
1999it [00:02, 969.32it/s]

736 users have an improved RR and 745 users have an degraded one.


In [15]:
# pipeModel

class pipeModel:
  def __init__(self, IMF = IMF, BPRMF = BPRMF):
    self.imf = IMF
    self.bprmf = BPRMF

  def predict(self, uid):
    df = pd.DataFrame(IMF.predict(uid)).rename(columns={0:'IMF'}).reset_index().merge(pd.DataFrame(BPRMF.predict(uid)).reset_index(), on='index').rename(columns={0:'BPRFM'})
    df.sort_values(by='BPRFM', inplace=True, ascending=False)
    df['new'] = (np.concatenate((df['IMF'][0:100].to_numpy(), np.zeros(len(df)-100))))
    df.sort_values(by='index', inplace=True)

    return df['new'].to_numpy()

mrr_score(pipeModel(), test_dataset, train=rating_dataset, k=100, verbose=True).mean()

1999it [00:20, 99.60it/s]


0.34594853369291295

In [16]:
linear_check = mrr_score(pipeModel(), test_dataset, train=rating_dataset, k=100, verbose=True) - mrr_score(BPRMF, test_dataset, train=rating_dataset, k=100, verbose=True)
print("%d users have an improved RR and %d users have an degraded one." %(np.count_nonzero(linear_check>0), np.count_nonzero(linear_check<0)))

1999it [00:20, 98.41it/s]
1999it [00:02, 967.28it/s]

710 users have an improved RR and 980 users have an degraded one.


In [17]:
#Now test your hybrid approaches for the quiz

test_Hybrid_a(linearModel())
test_Hybrid_b(pipeModel())


Hybrid a test case 0
445
Hybrid a test case 1
407
Hybrid b test case 0
16.40789222717285
Hybrid b test case 1
0.0


# Part-B. Analysing Recommendation Models

## Utility methods

Below, we provide a function, `get_top_K(model, uid : int, k : int)` which, when provided with a Spotlight model, will provide the top k predictions for the specified uid. The iids, their scores, and their embeddings are returned. 

In [18]:
from typing import Sequence, Tuple

def get_top_K(model, uid : int, k : int) -> Tuple[ Sequence[int], Sequence[float],  np.ndarray ] :
  #returns iids, their (normalised) scores in descending order, and item emebddings for the top k predictions of the given uid.

  from sklearn.preprocessing import minmax_scale

  from scipy.stats import rankdata
  # get scores from model
  scores = model.predict(uid)

  # map scores into rank 0..1 over the entire item space
  scores = minmax_scale(scores)

  #compute their ranks  
  ranks = rankdata(-scores)
  
  # get and filter iids, scores and embeddings
  rtr_scores = scores[ranks <= k]
  rtr_iids = np.argwhere(ranks <= k).flatten()
  if hasattr(model, '_net'):
    embs = model._net.item_embeddings.weight[rtr_iids]
  else:
    # not a model that has any embeddings
    embs = np.zeros([k,1])
  
  # identify correct ordering using numpy.argsort()
  ordering = (-1*rtr_scores).argsort()
  
  #return iids, scores and their embeddings in descending order of score
  return rtr_iids[ordering], rtr_scores[ordering], embs[ordering]

if BPRMF is not None:
  iids, scores, embs = get_top_K(BPRMF, 0, 10)
  print("Returned iids: %s" % str(iids))
  print("Returned scores: %s" % str(scores))
  print("Returned embeddings: %s" % str(embs))
else:
  print("You need to define BPRMF in Task 1")



Returned iids: [ 23 108  21  33   9  81  52 254  16   3]
Returned scores: [1.         0.9895131  0.9848315  0.92250896 0.9070817  0.90654314
 0.9005319  0.89310133 0.88378096 0.8836929 ]
Returned embeddings: tensor([[-0.0453,  1.3716, -0.8307, -1.2616,  1.6700,  1.0161,  1.1168,  2.3530,
         -1.2027,  0.8522, -1.0941, -0.6865, -0.5725, -2.0335, -1.2591,  0.6154,
         -0.1374, -1.6868, -1.8615, -0.7514,  1.9909, -0.3909,  1.9239,  1.3293,
         -1.2834, -0.4520,  1.1338,  0.3467,  2.5169, -2.1587,  1.2310,  1.1670],
        [ 0.1239,  1.1004,  0.0531, -1.1045,  1.9932,  1.5049,  1.0011,  1.9734,
         -1.6322, -0.8913, -0.6372,  0.7721, -1.1422, -2.2424, -1.1936, -0.5770,
          0.0762, -1.0283, -1.2807, -2.0889,  2.8154, -0.9600, -0.1419,  0.8408,
         -1.6067, -1.2905,  1.9169,  1.3988,  1.8646, -2.2028,  0.5365,  0.2022],
        [ 0.3845,  0.8188, -0.1892, -1.1793,  2.1731,  0.6669,  1.1271,  1.4538,
         -1.2173, -0.5447, -1.6713,  0.5249, -0.6132, -3.1082

## Task 3. Evaluation of Non-personalised Models
Implement the following four (non-personalised) baselines for ranking books based on their statistics:
 - Average rating, obtained from ratings_df, `ratings` column
 - Number of ratings, obtained from books_df (column `ratings_count`)
 - Number of 5* ratings, obtained from books_df (column `ratings_5`)
 - Fraction of 5* ratings, calculated from the two sources of evidence above, i.e (columns  `ratings_5` and `ratings_count`).

Evaluate these in terms of MRR using the provided test data. You may use the StaticModel class below. 

Hints: 
 - As in Exercise 2, the order of items returned by predict() is _critical_. You may wish to refer to iid_map.
 - For all models, you need to ensure that your values are not cast to ints. If you are extracting values from a Pandas series, it is advised to use [.astype(np.float32)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html).


In [19]:
class StaticModel:
  
  def __init__(self, staticscores):
    self.numitems = len(staticscores)
    #print(self.numitems)
    assert isinstance(staticscores, np.ndarray), "Expected a numpy array"
    assert staticscores.dtype == np.float32 or staticscores.dtype == np.float64, "Expected a numpy array of floats"
    self.staticscores = staticscores
  
  def predict(self, uid):
    #this model returns the same scores for each user    
    return self.staticscores

In [20]:
# Add your solution here
# a # average rating
scores_a = []
df_a = ratings_df[['book_id', 'rating']].groupby('book_id').agg(np.mean)
for i in range(len(df_a)):
  scores_a.append(df_a['rating'][iid_rev_map.get(i)])

model_a = StaticModel(np.array(scores_a))
mrr_score(model_a, test_dataset, train=rating_dataset, k=100, verbose=True).mean()

1999it [00:00, 2091.79it/s]


0.015052024168984034

In [21]:
# b # number of ratings
scores_b = []
df_b = books_df[['book_id', 'ratings_count']].set_index('book_id')
for i in range(len(df_b)):
  scores_b.append(df_b['ratings_count'][iid_rev_map.get(i)])

scores_b = minmax_scale(scores_b, feature_range=(0, 5))

model_b = StaticModel(np.array(scores_b))
mrr_score(model_b, test_dataset, train=rating_dataset, k=100, verbose=True).mean()

1999it [00:00, 2046.20it/s]


0.2396001188245477

In [22]:
# c # 5* ratings
scores_c = []
df_c = books_df[['book_id', 'ratings_5']].set_index('book_id')
for i in range(len(df_c)):
  scores_c.append(df_c['ratings_5'][iid_rev_map.get(i)])

scores_c = minmax_scale(scores_c, feature_range=(0, 5))

model_c = StaticModel(np.array(scores_c))
mrr_score(model_c, test_dataset, train=rating_dataset, k=100, verbose=True).mean()

1999it [00:00, 2119.03it/s]


0.2409670879930144

In [23]:
# d # fraction of 5* ratings
scores_d = []
df_d = books_df[['book_id', 'ratings_count', 'ratings_5']].set_index('book_id')
df_d['fraction'] = (df_d['ratings_5']/df_d['ratings_count'])
for i in range(len(df_d)):
  scores_d.append(df_d['fraction'][iid_rev_map.get(i)])

scores_d = minmax_scale(scores_d, feature_range=(0, 5))

model_d = StaticModel(np.array(scores_d))
mrr_score(model_d, test_dataset, train=rating_dataset, k=100, verbose=True).mean()

1999it [00:00, 2089.45it/s]


0.03415267465103555

## Task 4. Qualiatively Examining Recommendations

From now on, we will consider the `BPRMF` model.

In Recommender Systems, the ground truth (i.e. our list of books that the user has added to their "to_read" shelf) can be very incomplete. For instance, this can be because the user is not aware of the book yet.

For this reason, it is important to "eyeball" the recommendations, to understand what the system is surfacing, and whether the recommendations make sense. In this way, we understand if the recommendations are reasonable, even if they are for books that the user has not actually read according to the test dataset.

First, write a function, which given a uid (int), prints the *title and authors* of:
 - (a) the books that the user has previously shelved (c.f. `toread_dataset`)
 - (b) the books that the user will read in the future (c.f. `test_dataset`)
 - (c) the top 10 books that the user were recommended by `BPRMF` - you can make use of `get_top_K()`.

You can use the previously defined `getAuthorTitle()` function in your solution.
You will also want to compare books in (c) with those in (a) and (b).

Then, we will examine two specific users, namely uid 1805 (u336) and uid 179 (user u1331), to analyse if their recommendations make sense. Refer to the Task 4 quiz questions.


In [24]:
mrr_score(BPRMF, test_dataset, train=rating_dataset, k=100, verbose=True)[1805]

1999it [00:02, 941.60it/s]


0.05555555555555555

In [25]:
# Add your solution here
def eyeball(uid : int):
  # previously shelved
  print('=======================================================')
  print('Books that user uid %d has previously shelved:' % (uid))
  print('=======================================================')
  for i in range(len(toread_dataset)):
    if toread_dataset.user_ids[i] == uid:
      print(getAuthorTitle(toread_dataset.item_ids[i]))

  # to read in the future
  print('=======================================================')
  print('Books that user uid %d will read in the future:' % (uid))
  print('=======================================================')
  for i in range(len(test_dataset)):
    if test_dataset.user_ids[i] == uid:
      print(getAuthorTitle(test_dataset.item_ids[i]))

  # top 10 by BPRMF
  print('=======================================================')
  print('Top 10 books that user uid %d gets from BPRMF:' % (uid))
  print('=======================================================')
  iids, scores, embs = get_top_K(BPRMF, uid, 10)
  for i in range(len(iids)):
    print(getAuthorTitle(iids[i]))

  print('=======================================================')


In [26]:
eyeball(1805)

Books that user uid 1805 has previously shelved:
Stieg Larsson, Reg Keeland / The Girl Who Kicked the Hornet's Nest (Millennium, #3)
Suzanne Collins / Mockingjay (The Hunger Games, #3)
Dennis Lehane / Shutter Island
Suzanne Collins / Catching Fire (The Hunger Games, #2)
Paula Hawkins / The Girl on the Train
Robert Ludlum / The Bourne Supremacy (Jason Bourne, #2)
John Grisham / The Client
Thomas Harris / The Silence of the Lambs  (Hannibal Lecter, #2)
Daphne du Maurier, Sally Beauman / Rebecca
Robert Ludlum / The Bourne Identity (Jason Bourne, #1)
Robert Galbraith, J.K. Rowling / The Cuckoo's Calling (Cormoran Strike, #1)
Stephen King / Misery
Michael Crichton / Jurassic Park (Jurassic Park, #1)
Robert Ludlum / The Bourne Ultimatum (Jason Bourne, #3)
Stephen King, Bernie Wrightson / The Stand
Michael Crichton / The Andromeda Strain
Thomas Harris / Red Dragon (Hannibal Lecter, #1)
Lee Child / Die Trying (Jack Reacher, #2)
Lee Child / Worth Dying For (Jack Reacher, #15)
Lee Child / Tripwi

In [27]:
eyeball(179)

Books that user uid 179 has previously shelved:
Dan Brown / Angels & Demons  (Robert Langdon, #1)
Suzanne Collins / The Hunger Games (The Hunger Games, #1)
Antoine de Saint-Exupéry, Richard Howard, Dom Marcos Barbosa, Melina Karakosta / The Little Prince
Truman Capote / Breakfast at Tiffany's
Dan Brown / The Da Vinci Code (Robert Langdon, #2)
Laura Ingalls Wilder, Garth Williams / Little House on the Prairie (Little House, #2)
Milan Kundera, Michael Henry Heim / The Unbearable Lightness of Being
Suzanne Collins / Catching Fire (The Hunger Games, #2)
John Grisham / The Client
J.R.R. Tolkien / The Lord of the Rings (The Lord of the Rings, #1-3)
J.R.R. Tolkien / The Hobbit
Margaret Mitchell / Gone with the Wind
Neil Gaiman / Stardust
Laura Ingalls Wilder, Garth Williams / Little House in the Big Woods (Little House, #1)
Pearl S. Buck / The Good Earth (House of Earth, #1)
Dan Brown / Digital Fortress
Daniel Keyes / Flowers for Algernon
Neil Gaiman / Coraline
Dan Brown / Deception Point
Joh

# Part-C. Diversity of Recommendations

This part of the exercise is concerned with diversification, as covered in Lecture 11.

## Task 5. Measuring Intra-List Diversity


For the BPR implicit factorisation model, implement the Intra-list diversity measure (see Lecture 11) of the top 5 scored items based on their item embeddings in the `BPRMF` model. 

Implement your ILD as a function with the specification:
```python
def measure_ild(top_books : Sequence[int], K : int=5) -> float
```
where:
 - `top_books` is a list or a Numpy array of iids that have been returned for a particular user. For instance, it can be obtained from `get_top_K()`.
 - `K` is the number of top-ranked items to consider from `top_books`. 
 - Your implementation should use the item emebddings stored in the `BPRMF` model.

Calculate the ILD (with k=5). Using your code for Task 4, identify the books previously shelved and recommended for the specific users requested in the quiz, and use these to analyse the recommendations.

Hints:
 - As can be seen in `get_top_K()`, item embeddings can be obtained from `BPRMF._net.item_embeddings.weight[iid]`.
 - For obtaining the cosine similarity of PyTorch tensors, use `nn.functional.cosine_similarity(, , axis=0)`.


In [28]:
# Add your solution here
import torch.nn as nn

def measure_ild(top_books : Sequence[int], K : int=5) -> float:
  ILD = 0.0
  
  for i in range(K):
    for j in range(K):
      if i != j:
        ILD += (1-nn.functional.cosine_similarity(
            BPRMF._net.item_embeddings.weight[top_books[i]], 
            BPRMF._net.item_embeddings.weight[top_books[j]], 
            axis=0).item())

  ILD = ILD / (K*(K-1))
  
  return ILD

In [29]:
iids_u336, scores_u336, embs_u336 = get_top_K(BPRMF, 1805, 5)
iids_u1331, scores_u1331, embs_u1331 = get_top_K(BPRMF, 179, 5)

In [30]:
measure_ild(iids_u336)

0.7484898872673511

In [31]:
for i in range(len(iids_u336)):
  print(getAuthorTitle(iids_u336[i]))

Suzanne Collins / The Hunger Games (The Hunger Games, #1)
Dan Brown / The Da Vinci Code (Robert Langdon, #2)
Dan Brown / The Lost Symbol (Robert Langdon, #3)
Michael Crichton / Disclosure
George R.R. Martin / A Clash of Kings  (A Song of Ice and Fire, #2)


In [32]:
measure_ild(iids_u1331)

0.27872883677482607

In [33]:
for i in range(len(iids_u1331)):
    print(getAuthorTitle(iids_u1331[i]))

John Grisham / The Partner
John Grisham / The Pelican Brief
John Grisham / The Client
John Grisham / The Brethren
John Grisham / The Street Lawyer


## Task 6. Implement MMR Diversification 

Develop an Maximal Marginal Relevance (**MMR**) diversification technique, to re-rank the top-ranked recommendations for a given user.

Your function should adhere to the specification as follows:
```python
def mmr(iids : Sequence[int], scores : Sequence[float], embs : np.ndarray, alpha : float) -> Sequence[int]:
```

where iids is a list of iids, scores are their corresponding scores (in descending order), embs is their embeddings, and alpha controls the diversification tradeoff. The function returns a re-ordering of iids. As in previous Exercises, type hints are provided for clarity; a Sequence can be a list or numpy array. 

Hints:
 - As above, for obtaining the cosine similarity of PyTorch tensors, use nn.functional.cosine_similarity(, , axis=0).

To use your `mmr()` function, provide it with the outputs of `get_top_K()`. For example, to obtain an MMR reordering of the top 10 predictions of uid 0, we can run:
```
mmr( *get_top_K(bprmodel, 0, 10), 0.5)
```

Thereafter, we provide test cases for your MMR implementation, which you  should report in the quiz. We also ask for the ILD values before and after the application of MMR.


In [34]:
from typing import Sequence
def mmr(iids : Sequence[int], scores : Sequence[float], embs : np.ndarray, alpha : float) -> Sequence[int]:

  assert len(iids) == len(scores)
  assert len(iids) == embs.shape[0]
  assert len(embs.size()) == 2

  iids = np.array(iids)
  rtr_iids = []
  for i in range(len(iids)):
    if len(rtr_iids) == 0:
      rtr_iids.append(iids[np.argmax(scores)])
    else:
      mmrs = -np.ones(len(iids))
      for j in range(len(iids)): 
        if iids[j] not in rtr_iids:
          front = alpha * scores[j]
          sims = -np.ones(len(rtr_iids))
          for p in range(len(rtr_iids)):
            a = np.where(iids==rtr_iids[p])[0][0]
            sims[p] = nn.functional.cosine_similarity(embs[j], embs[a], dim=0).item()
          back = (1-alpha) * sims.max()
          mmrs[j] = front - back

      rtr_iids.append(iids[np.argmax(mmrs)])
  
  #input your solution here returns a re-ordering of iids, such that the first ranked item is first in the list

  return rtr_iids

In [35]:
def run_MMR_testcases(mmrfn):
  example_embeddings1 = torch.tensor([[1.0,1.0],[1.0,1.0],[0,1.0],[0.1, 1.0]])
  example_embeddings2 = torch.tensor([[1.0,1.0],[1.0,1.0],[0.02,1.0],[0.01,1.0]])
  print("Testcase 0 : %s" % mmrfn([1,2,3,4], [0.5, 0.5, 0.5, 0.5],  example_embeddings1, 0.5)[0] )
  print("Testcase 1 : %s" % mmrfn([1,2,3,4], [0.5, 0.5, 0.5, 0.5],  example_embeddings1, 0.5)[1] )
  print("Testcase 2 : %s" % mmrfn([1,2,3,4], [4, 3, 2, 1],  example_embeddings1, 1)[1] )
  print("Testcase 3 : %s" % mmrfn([1,2,3,4], [0.99, 0.98, 0.97, 0.001],  example_embeddings2, 0.001)[1] )
  print("Testcase 4 : %s" % mmrfn([1,2,3,4], [0.99, 0.98, 0.97, 0.001],  example_embeddings2, 0.5)[1] )

run_MMR_testcases(mmr)

Testcase 0 : 1
Testcase 1 : 3
Testcase 2 : 2
Testcase 3 : 4
Testcase 4 : 3


In [36]:
mmr( *get_top_K(BPRMF, 179 , 5), 0.5)

[89, 88, 391, 906, 92]

Now we can analyse the impact of our MMR implementation. Let's consider again uid 179 (user u1331). 

Apply MMR on the top 10 results obtained from the BPRMF model using `get_top_K()`, with an alpha value of 0.5. The following code should help:
```python
mmr( *get_top_K(bprmodel, 179, 10), 0.5)
```

Finally, anayse the returned books. Calculate the ILD (with `k=5`), and examine the authors and titles (using `getAuthorTitle()`). 

Now answer the questions in Task 6 of the Moodle quiz.


In [37]:
#add your solution here
measure_ild(mmr( *get_top_K(BPRMF, 179, 10), 0.5), K=5)

0.5566449973732233

In [38]:
new = mmr( *get_top_K(BPRMF, 179, 10), 0.5)[:5]
for i in range(len(new)):
    print(getAuthorTitle(new[i]))

John Grisham / The Partner
J.K. Rowling, Mary GrandPré / Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
John Grisham / The Street Lawyer
John Grisham / The Pelican Brief
John Grisham / The Brethren


In [39]:
new[1]

9

# Task 7

This task is not a practical task - instead there are questions that tests your understanding of some related content of the course in the quiz.

# End of Exercise

As part of your submission, you should complete the Exercise 3 quiz on Moodle.
You will need to upload your notebook, complete with the **results** of executing the code.